# Exemplo de como fazer um crawler utilizando urllib e BeatifulSoup

## Importações necessárias

In [ ]:
# importações
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import os

## Capturar conteúdo HTML da página (GET)

In [ ]:
# fazer GET da página web
init = "https://pt.wikipedia.org/wiki/Cerveja"
get = urlopen(init)
# recuperar conteúdo HTML
html = get.read()

In [ ]:
# mostrar html
html

## Procurar por links (marcador <a\>) no conteúdo da página

In [ ]:
# criar objeto do BS
bsObj = BeautifulSoup(html, "html.parser")
# encontrar todos os links <a>
links_a = bsObj('a')


In [ ]:
# mostral links extraídos
links_a

## Procurar apenas links com href válidos (não vazios e que não comecem com #)

In [ ]:
# utilizar beatiful soup para pegar apenas os links válidos
# função para verificar se o link é válido (não vazio e não inicia com #)
def links_validos(href):
    return href and href[0] != '#'
# função é enviada como valor do parâmtero href
validos = bsObj('a', href=links_validos)

In [ ]:
validos

## Outros exemplos para encontrar elementos

In [ ]:
# procurar links com filtro do atributo class
filtro_classe = bsObj('a', attrs={'class':'mw-redirect'})
filtro_classe

In [ ]:
# procurar links com filtro do atributo class e title
filtro_classe_titulo = bsObj('a', attrs={'class':'mw-redirect', 'title':'Pilsen'})
filtro_classe_titulo

In [ ]:
# procurar links com recursive = False
filtro_recursive = bsObj('a', recursive=False)
filtro_recursive

In [ ]:
# procurar links com texto Pilsen
filtro_string = bsObj('a', string='Pilsen')
filtro_string

In [ ]:
# procurar links utilizando **kwargs
filtro_classe = bsObj('a', class_='mw-redirect')
filtro_classe_titulo = bsObj('a', class_='mw-redirect', title='Pilsen')
filtro_classe, filtro_classe_titulo


In [ ]:
# procurar por elementos do tipo span com classe mw-headline
filtro_span = bsObj('span', class_='mw-headline')
# procurar por qualquer elemento que contenha o texto cerveja
# retorna apenas o texto, e não o elemento
import re
filtro_cerveja = bsObj(string=re.compile('cerveja',re.IGNORECASE))
filtro_span, filtro_cerveja

## Extrair os atributos href dos links HTML e transformar nos endereços completos

In [ ]:
# retornar apenas atributo href (onde está o link)
# e transformar no endereço completo utilizando urljoin 
# urljoin mescla dois links
# se for um link para outro endereço, utiliza apenas o novo link
# se for um link para o mesmo endereço, junta o endereço base com o link
# exemplo: urljoin("https://pt.wikipedia.org/wiki/Cerveja", "/wiki/Bebida") --> https://pt.wikipedia.org/wiki/Bebida
# exemplo: urljoin("https://pt.wikipedia.org/wiki/Cerveja", "http://google.com") --> http://google.com
enderecos = [urljoin(init, l.get('href')) for l in links_a]

In [ ]:
# mostrar endereços dos links
enderecos

In [ ]:
# apenas endereços que comecem com https://pt.wikipedia.org/wiki/
enderecos = [e for e in enderecos if e.startswith('https://pt.wikipedia.org/wiki')]

In [ ]:
enderecos

## Percorrer lista de links para armazenar o conteúdo
## exemplo para os 10 primeiros links da lista

In [ ]:
# percorrer os 10 primeiros links para fazer fazer o GET e adicionar em uma lista
lista_html = []
for e in enderecos_wiki[:10]:
    # retorna um objeto HTTPResponse
    get = urlopen(str(e))
    # retorna o conteudo html
    html = get.read()
    # coloca em uma lista
    lista_html.append(html)

In [ ]:
# quantidade de páginas retornadas
lista_html

## Salvar lista de conteúdos em arquivos

In [ ]:
# salvar html em um arquivos
for i,h in enumerate(lista_html):
    with open('pagina_'+str(i)+'.txt','wb') as file:
        file.write(h)

# Juntando tudo: crawler para recuperar páginas da wikipedia com tamanho máximo de páginas

In [ ]:
# função para encontrar os links
def busca_links(conteudo, link):
    links = BeautifulSoup(conteudo, "html.parser")('a', href=links_validos)
    # retorna o link absoluto
    links = [wiki for wiki in [urljoin(init, l.get('href')) for l in links] if wiki.startswith('https://pt.wikipedia.org/wiki') ]
    return links

In [ ]:
# função para salvar link em um arquivo
def salva_arquivo(nome, conteudo):
    with open(nome,'w', encoding="utf8") as file:
        file.write(conteudo)

In [ ]:
# função para fazer o get
def crawler(init, max = 10):
    buscar = set([init])
    buscados = set()
    contador = 1
    while buscar and contador <= max:
        # remove último elemento do vetor
        link = buscar.pop()
        if link not in buscados:
            print(link)
            try: 
                conteudo = urlopen(link).read().decode('utf-8')
                salva_arquivo('wikis/wiki'+str(contador)+'.txt', conteudo)
                contador += 1            
                novos_links = busca_links(conteudo, link)
                # atualiza lista para buscar com novos links.
                # o método update do set desconsidera os repetidos os repetidos
                buscar.update(novos_links)
                buscados.add(link)
            except Exception as ex:
                print(ex)
                pass
  

In [ ]:
# executar crawler
init = 'https://pt.wikipedia.org/wiki/Cerveja'
r = crawler(init, max = 10)

## ler documentos do arquivo para extrair o texto e salvar em um novo arquivo

In [ ]:
# função para remover espaços e quebra de linha
def remove_espacos(texto) :
    linhas = (linha.strip() for linha in texto.splitlines())
    linhas = (frase.strip() for linha in linhas for frase in linha.split("    "))
    return ' '.join(linha for linha in linhas if linha)

In [ ]:
# percorrer os arquivos da pasta onde estão os arquivos
for i, arquivo in enumerate(os.listdir(os.getcwd()+'/wikis')):
    with open('wikis/'+arquivo, 'r', encoding="utf8") as arq:
        # le conteúdo do arquivo
        html = arq.read()
    bsObj = BeautifulSoup(html, "html.parser")       
    # remover tags de script e style
    for remove in bsObj(["script", "style"]): remove.extract()
    texto = bsObj.text
    # salvar em um novo arquivo
    salva_arquivo('wikis_text/wiki'+str(i+1)+'.txt', texto)        
    

In [ ]:
# ler documentos do arquivo para extrair o texto apenas da div com id content
# e salvar em um novo arquivo
lista_conteudos = []
# percorrer os arquivos da pasta onde estão os arquivos
for i, arquivo in enumerate(os.listdir(os.getcwd()+'/wikis')):
    with open('wikis/'+arquivo, 'r', encoding="utf8") as arq:
        # le conteúdo do arquivo
        html = arq.read()
    bsObj = BeautifulSoup(html, "html.parser")       
    # remover tags de script e style
    for remove in bsObj(["script", "style"]): remove.extract()    
    # extrair o texto apenas da div com id content
    texto = bsObj('div', id='content')[0].text
    # salvar em um novo arquivo
    salva_arquivo('wikis_text/wiki'+str(i+1)+'.txt', texto)        
    